# Week 23: Regulation & Compliance in Algorithmic Trading

## 🎯 Learning Objectives

By the end of this week, you will understand:
- **Regulatory Framework**: SEC, FINRA, MiFID II
- **Market Manipulation**: What to avoid
- **Model Risk Management**: SR 11-7 guidelines
- **Documentation**: Audit trails and compliance

---

## Why Compliance Matters?

- Legal consequences (fines, jail)
- Reputation risk
- Career implications
- Ethical responsibility

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("✅ Libraries loaded!")
print("📚 Week 23: Regulation & Compliance")

---

## Part 1: Regulatory Framework

### US Regulations

| Regulator | Scope | Key Rules |
|-----------|-------|----------|
| SEC | Securities | Reg SHO, Reg NMS |
| FINRA | Broker-dealers | Rule 3110, 5310 |
| CFTC | Derivatives | Reg AT (proposed) |

### EU Regulations

- **MiFID II**: Algo trading requirements
- **MAR**: Market abuse prevention

### Key Requirements

1. Pre-trade risk controls
2. Kill switches
3. Audit trails
4. Testing requirements

In [ ]:
class ComplianceChecker:
    """Check trading activity for compliance issues"""
    
    def __init__(self):
        self.violations = []
    
    def check_wash_trading(self, trades):
        """Detect potential wash trading"""
        # Wash trading: Buy and sell same security with no change in position
        for i in range(len(trades) - 1):
            if (trades[i]['symbol'] == trades[i+1]['symbol'] and
                trades[i]['side'] != trades[i+1]['side'] and
                abs(trades[i]['quantity'] - trades[i+1]['quantity']) < 10 and
                abs((trades[i+1]['timestamp'] - trades[i]['timestamp']).total_seconds()) < 60):
                self.violations.append({
                    'type': 'POTENTIAL_WASH_TRADE',
                    'trades': [trades[i], trades[i+1]]
                })
    
    def check_spoofing(self, orders, executions):
        """Detect potential spoofing"""
        # Spoofing: Place orders with intent to cancel before execution
        cancel_rate = 1 - len(executions) / max(len(orders), 1)
        if cancel_rate > 0.90:  # >90% cancel rate is suspicious
            self.violations.append({
                'type': 'HIGH_CANCEL_RATE',
                'cancel_rate': cancel_rate
            })
    
    def check_concentration(self, positions, threshold=0.10):
        """Check for excessive position concentration"""
        total = sum(abs(p) for p in positions.values())
        for symbol, position in positions.items():
            if abs(position) / total > threshold:
                self.violations.append({
                    'type': 'CONCENTRATION_RISK',
                    'symbol': symbol,
                    'percentage': abs(position) / total
                })

# Example usage
checker = ComplianceChecker()

# Simulated trades
trades = [
    {'symbol': 'AAPL', 'side': 'BUY', 'quantity': 1000, 'timestamp': datetime(2024, 1, 1, 9, 30, 0)},
    {'symbol': 'AAPL', 'side': 'SELL', 'quantity': 1000, 'timestamp': datetime(2024, 1, 1, 9, 30, 30)},  # Suspicious
    {'symbol': 'MSFT', 'side': 'BUY', 'quantity': 500, 'timestamp': datetime(2024, 1, 1, 10, 0, 0)},
]

checker.check_wash_trading(trades)

print("Compliance Check Results")
print("="*50)
if checker.violations:
    for v in checker.violations:
        print(f"⚠️ {v['type']}")
else:
    print("✅ No violations detected")

---

## Part 2: Market Manipulation

### Prohibited Activities

| Type | Description | Consequence |
|------|-------------|-------------|
| Spoofing | Fake orders to move prices | Criminal |
| Layering | Multiple fake orders | Criminal |
| Wash Trading | Trade with yourself | Criminal |
| Front-running | Trade ahead of clients | Criminal |
| Pump & Dump | Inflate then sell | Criminal |

In [ ]:
def detect_layering(order_book_snapshots):
    """Detect potential layering behavior"""
    alerts = []
    
    for i in range(1, len(order_book_snapshots)):
        prev = order_book_snapshots[i-1]
        curr = order_book_snapshots[i]
        
        # Check if many orders appeared and disappeared
        bid_change = abs(curr['bid_depth'] - prev['bid_depth'])
        ask_change = abs(curr['ask_depth'] - prev['ask_depth'])
        
        # Large depth changes without price movement is suspicious
        if (bid_change > 5000 or ask_change > 5000) and abs(curr['mid'] - prev['mid']) < 0.01:
            alerts.append({
                'time': curr['timestamp'],
                'type': 'POTENTIAL_LAYERING',
                'bid_change': bid_change,
                'ask_change': ask_change
            })
    
    return alerts

# Simulated order book
np.random.seed(42)
order_book = []
mid_price = 100

for i in range(100):
    mid_price += np.random.randn() * 0.1
    bid_depth = np.random.randint(1000, 3000)
    ask_depth = np.random.randint(1000, 3000)
    
    # Inject suspicious activity
    if i == 50:
        bid_depth = 10000  # Suspicious spike
    if i == 51:
        bid_depth = 1500   # Back to normal
    
    order_book.append({
        'timestamp': i,
        'mid': mid_price,
        'bid_depth': bid_depth,
        'ask_depth': ask_depth
    })

alerts = detect_layering(order_book)
print(f"Detected {len(alerts)} potential layering incidents")
for alert in alerts:
    print(f"  Time {alert['time']}: Bid change = {alert['bid_change']}")

---

## Part 3: Model Risk Management (SR 11-7)

### Key Requirements

1. **Model Development**: Documentation, assumptions
2. **Model Validation**: Independent review
3. **Model Use**: Appropriate application
4. **Model Monitoring**: Ongoing performance

### Documentation Required

- Model purpose and scope
- Methodology and assumptions
- Data sources and quality
- Limitations and known issues
- Validation results

In [ ]:
class ModelInventory:
    """Track and document all models per SR 11-7"""
    
    def __init__(self):
        self.models = []
    
    def register_model(self, model_info):
        required_fields = [
            'name', 'purpose', 'methodology', 'data_sources',
            'assumptions', 'limitations', 'owner', 'validator',
            'last_validation', 'next_validation'
        ]
        
        missing = [f for f in required_fields if f not in model_info]
        if missing:
            raise ValueError(f"Missing required fields: {missing}")
        
        model_info['registration_date'] = datetime.now()
        self.models.append(model_info)
        return len(self.models)
    
    def get_models_due_validation(self):
        return [m for m in self.models 
                if m['next_validation'] <= datetime.now()]

# Example model registration
inventory = ModelInventory()

model = {
    'name': 'Momentum Alpha Model',
    'purpose': 'Generate equity trading signals based on price momentum',
    'methodology': 'Cross-sectional momentum with 12-1 lookback',
    'data_sources': ['CRSP daily prices', 'Compustat fundamentals'],
    'assumptions': ['Markets are not fully efficient', 'Past returns predict future'],
    'limitations': ['Underperforms in reversals', 'High turnover'],
    'owner': 'Quant Research Team',
    'validator': 'Model Risk Management',
    'last_validation': datetime(2024, 1, 1),
    'next_validation': datetime(2025, 1, 1)
}

model_id = inventory.register_model(model)
print(f"Model registered with ID: {model_id}")
print(f"\nModel Documentation:")
for k, v in model.items():
    print(f"  {k}: {v}")

---

## Part 4: Audit Trail & Logging

### Requirements

- Complete order lifecycle
- Timestamps (microsecond precision)
- Decision logic
- Immutable records

In [ ]:
class AuditLogger:
    """Compliance-ready audit logging"""
    
    def __init__(self):
        self.logs = []
    
    def log_order_event(self, event_type, order_id, details):
        entry = {
            'timestamp': datetime.utcnow().isoformat(),
            'event_type': event_type,
            'order_id': order_id,
            'details': details
        }
        self.logs.append(entry)
        return entry
    
    def log_model_decision(self, model_name, inputs, outputs, rationale):
        entry = {
            'timestamp': datetime.utcnow().isoformat(),
            'event_type': 'MODEL_DECISION',
            'model_name': model_name,
            'inputs': inputs,
            'outputs': outputs,
            'rationale': rationale
        }
        self.logs.append(entry)
        return entry
    
    def reconstruct_order_lifecycle(self, order_id):
        return [l for l in self.logs if l.get('order_id') == order_id]

# Example usage
logger = AuditLogger()

# Log model decision
logger.log_model_decision(
    model_name='MomentumAlpha',
    inputs={'symbol': 'AAPL', 'momentum_score': 0.85},
    outputs={'signal': 'BUY', 'confidence': 0.75},
    rationale='Strong 12-month momentum above threshold'
)

# Log order lifecycle
order_id = 'ORD-001'
logger.log_order_event('ORDER_CREATED', order_id, {'symbol': 'AAPL', 'side': 'BUY', 'qty': 1000})
logger.log_order_event('ORDER_SUBMITTED', order_id, {'venue': 'NYSE', 'price': 150.00})
logger.log_order_event('ORDER_FILLED', order_id, {'fill_price': 150.05, 'fill_qty': 1000})

print("Audit Log Entries:")
print("="*60)
for log in logger.logs:
    print(f"{log['timestamp']}: {log['event_type']}")
    if 'details' in log:
        print(f"  Details: {log['details']}")

---

## Interview Questions

### Conceptual
1. What is spoofing and why is it illegal?
2. What's the difference between front-running and legitimate trading?
3. Why is model documentation important for compliance?

### Technical
1. How would you implement a kill switch?
2. What data do you need to store for audit purposes?
3. How do you detect wash trading algorithmically?

### Finance-Specific
1. What are the key MiFID II requirements for algo trading?
2. How does SR 11-7 apply to trading models?
3. What's the role of model validation?

---

## Key Takeaways

| Area | Requirement | Implementation |
|------|-------------|---------------|
| Pre-trade | Risk controls | Position limits, price checks |
| Audit | Record keeping | Immutable logs, timestamps |
| Model | Documentation | SR 11-7 compliance |
| Testing | Pre-deployment | Simulation, UAT |